In [1]:
from utils.RobotConfig import RobotConfig
from utils.ObstacleSpheres import ObsConfig
import casadi
import pinocchio as pin
from pinocchio import casadi as cpin
import numpy as np

In [2]:
Ntp = 10

In [3]:
config_file = 'config/SimpleConfig.yaml'
q_waypoints = np.array([[0, np.pi/4, 0, 0, 0, 0], 
                    [-np.pi/6, np.pi/12, np.pi/6, -np.pi/6, -np.pi/3, -np.pi/3],
                    [-np.pi/4, np.pi/4, np.pi/3, 0, -np.pi/6, -np.pi/2]])

obs = ObsConfig(config_file)
obs.spheres_from_robot(q0 = q_waypoints[0].astype(float).tolist())  # Get initial robot obstacle spheres
obs.spheres_from_env()

Robot = RobotConfig(config_file)
Robot.homePosition = q_waypoints[0]
ngc = Robot.model.nq   # Number of joints
ndof = Robot.model.nv   # Number of DOF
nx = 2*ngc    # Number of states
cx = casadi.SX.sym("x", 2*ngc, 1)
ctau = casadi.SX.sym("tau", ndof, 1)
cacc = cpin.aba(Robot.cmodel, Robot.cdata, cx[:ngc], cx[ngc:], ctau) # Forward dynamics

# Joint limits
Robot.lowerVelocityLimit = np.pi/2 * np.array([-1, -1, -1, -1, -1, -1])  # rad/s
Robot.upperVelocityLimit = np.pi/2 * np.array([1, 1, 1, 1, 1, 1])
Robot.lowerTorqueLimit = 100 * np.array([-1, -1, -1, -1, -1, -1]) 
Robot.upperTorqueLimit = 100 * np.array([1, 1, 1, 1, 1, 1])

q0 = q_waypoints[0]
qT = q_waypoints[-1]
tau0 = pin.rnea(Robot.model, Robot.data, q0, np.zeros(ndof), np.zeros(ndof))
tauT = pin.rnea(Robot.model, Robot.data, qT, np.zeros(ndof), np.zeros(ndof))
print(tau0)
print(tauT)

Feq = casadi.Function(
    "Dyn",
    [cx, ctau],
    [casadi.vertcat(cx[ngc:], cacc)],
)

print(Robot.forward_kinematics(q0))
print(Robot.forward_kinematics(qT))

[  0.         -36.08572402  -9.26563407   0.97299901  -0.09997149
   0.        ]
[ 8.88178420e-16 -2.17616448e+01  5.05844512e+00  1.31084531e+00
 -1.18267539e-01  0.00000000e+00]
(array([[-7.07106781e-01,  7.07106781e-01,  3.58690743e-17,
         5.07349116e-01],
       [ 1.45030033e-10,  1.45029982e-10,  1.00000000e+00,
         2.32900000e-01],
       [ 7.07106781e-01,  7.07106781e-01, -2.05103403e-10,
        -4.85846208e-01],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00]]), array([[-7.07106781e-01,  7.07106781e-01,  3.58690743e-17],
       [ 1.45030033e-10,  1.45029982e-10,  1.00000000e+00],
       [ 7.07106781e-01,  7.07106781e-01, -2.05103403e-10]]), array([ 0.50734912,  0.2329    , -0.48584621]))
(array([[-0.6830127 , -0.19505974,  0.70387879,  0.23698971],
       [ 0.6830127 , -0.51204704,  0.52086608,  0.07350954],
       [ 0.25881905,  0.8365163 ,  0.48296291, -0.44294913],
       [ 0.        ,  0.        ,  0.        ,  1.        ]]), 

### Collision Sphere Generation

In [4]:
cq = casadi.SX.sym('q', ngc)
get_obstacle_distances = casadi.Function(
    'get_obstacle_distances', 
    [cq], 
    [casadi.vertcat(*Robot.obstacle_distances(cq, obs.env_spheres, obs.robot_spheres))]
    )

# test
print(get_obstacle_distances(q_waypoints[0]))
dis = Robot.obstacle_distances(q_waypoints[0], obs.env_spheres, obs.robot_spheres)
print(Robot.collision_status(q_waypoints[0], obs.env_spheres, obs.robot_spheres))  # False is not collision

[0.453906, 0.418906, 0.0872761, 0.143539, 0.458041, 0.402783, 0.311941, 0.292739, 0.176466, 0.19761, 0.109124, 0.155739, 0.230739, 0.235196, 0.411669, 0.371751, 0.478139, 0.415691, 0.0600583, 0.145826, 0.0192186, 0.215387, 0.0199047, 0.142437, 0.00449313, 0.151433, 0.00495637, 0.181046, 0.0401787, 0.145106, 0.00189849, 0.165443, 0.022891, 0.228222, 0.0101919, 0.144686, 0.0379068, 0.214486, 0.029066, 0.214245, 0.0478279, 0.238846, 0.0379479, 0.206887, 0.0247187, 0.145665, 0.0294815, 0.146827, 0.0291062, 0.167768, 0.0198831, 0.14398, 0.0515788, 0.156522]
False


In [5]:
from utils.Lagrange import *
from utils.LGL_collocation import LGL_collocation
from utils.LocalChart import *

Np = q_waypoints.shape[0]-1  # Number of phase (dersired waypoints: N+1)
T = Np*Ntp  # Number of total collocation points / optimization variables

LGL_Pts, LGL_Wts = Normalize_LGL_Pts_Wts(Ntp)
DL,_ = differentiation_matrix(Ntp, LGL_Pts)

LGL = LGL_collocation(Ntp, ngc, ndof)
StatesDiff, StateDiff, LocalsDiff, LocalDiff= LGL.getDiffFuncs()

### Initial Guesses

In [6]:
t_initial = 1
ini_collo_pts = t_initial/2 * (LGL_Pts+1)
ini_q = np.full((Ntp, ngc), 0.0)
ini_x = [None] * T
ini_u = [None] * T


for k in range(Np):
    for j in range(Ntp):
        ini_q[j,:] = q_waypoints[k] + ini_collo_pts[j]/t_initial * (q_waypoints[k+1]-q_waypoints[k])

    ini_dq = np.array([np.squeeze(StateDiff(casadi.vertcat(*ini_q), t_initial, DL[i,:])) for i in range(Ntp)])
    ini_ddq = np.array([np.squeeze(StateDiff(casadi.vertcat(*ini_dq), t_initial, DL[i,:])) for i in range(Ntp)])
    ini_tau = np.array([pin.rnea(Robot.model, Robot.data, ini_q[i], ini_dq[i], ini_ddq[i]) for i in range(Ntp)])  # M@ddq + C@dq + G
    
    for j in range(Ntp):
        t = k*Ntp + j
        ini_x[t] =  np.concatenate([ini_q[j],ini_dq[j]])
        ini_u[t] = np.array(ini_tau[j])
        print(f"Collision State {t}: {Robot.collision_status(ini_q[j], obs.env_spheres, obs.robot_spheres)}")

Collision State 0: False
Collision State 1: False
Collision State 2: False
Collision State 3: False
Collision State 4: False
Collision State 5: False
Collision State 6: False
Collision State 7: False
Collision State 8: False
Collision State 9: False
Collision State 10: False
Collision State 11: False
Collision State 12: False
Collision State 13: False
Collision State 14: False
Collision State 15: True
Collision State 16: False
Collision State 17: False
Collision State 18: True
Collision State 19: True


### Nonlinear Programming Problem (NLP)

In [7]:
opti = casadi.Opti()
# Optimization variables
var_xs = [opti.variable(2*ngc) for _ in range(T)]
var_us = [opti.variable(ndof) for _ in range(T)]
var_deltaT = [opti.variable(1) for _ in range(Np)]

# Constraints
for i in range(Np):
    opti.subject_to(var_deltaT[i] >= 0.0)

    for tp in range(Ntp):
        t = i*Ntp + tp
        # dynamics
        opti.subject_to(StatesDiff(casadi.vertcat(*var_xs[i*Ntp: (i+1)*Ntp]), var_deltaT[i], DL[tp,:]) 
                        == Feq(var_xs[t], var_us[t]))
        
        # joint constraints
        opti.subject_to(var_xs[t][:ngc] <= Robot.upperPositionLimit)
        opti.subject_to(var_xs[t][:ngc] >= Robot.lowerPositionLimit)
        # joint velocity constraints
        opti.subject_to(var_xs[t][ngc:] <= Robot.upperVelocityLimit)
        opti.subject_to(var_xs[t][ngc:] >= Robot.lowerVelocityLimit)
        # torque constraints
        opti.subject_to(var_us[t] <= Robot.upperTorqueLimit)
        opti.subject_to(var_us[t] >= Robot.lowerTorqueLimit)

        # collision avoidance
        opti.subject_to(get_obstacle_distances(var_xs[t][:ngc]) >= 0.0)

# waypoints constraints (N-1 middle waypoints)
for i in range(1, Np):
    opti.subject_to(var_xs[i*Ntp][:ngc] == q_waypoints[i])

# continuity
for i in range(1, Np):
    opti.subject_to(var_xs[i*Ntp-1] == var_xs[i*Ntp])
    opti.subject_to(var_us[i*Ntp-1] == var_us[i*Ntp])

# continuity of derivatives
for i in range(Np-1):
    opti.subject_to(StatesDiff(casadi.vertcat(*var_xs[i*Ntp: (i+1)*Ntp]), var_deltaT[i], DL[Ntp-1,:]) 
                    == StatesDiff(casadi.vertcat(*var_xs[(i+1)*Ntp: (i+2)*Ntp]), var_deltaT[i+1], DL[0,:]))
    
    opti.subject_to(LocalDiff(casadi.vertcat(*var_us[i*Ntp: (i+1)*Ntp]), var_deltaT[i], DL[Ntp-1,:])
                   == LocalDiff(casadi.vertcat(*var_us[(i+1)*Ntp: (i+2)*Ntp]), var_deltaT[i], DL[0,:]))


# boundary conditions
opti.subject_to(var_xs[0][:ngc] == q0)
opti.subject_to(var_xs[-1][:ngc] == qT)
opti.subject_to(var_xs[0][ngc:] == 0)
opti.subject_to(var_xs[-1][ngc:] == 0)
#opti.subject_to(var_us[0] == tau0)
#opti.subject_to(var_us[-1] == tauT)

# Objective
totalcost = 0

R = np.eye(ndof)
for i in range(Np):
    for tp in range(Ntp):
        var_xs_Phase = var_xs[i*Ntp: (i+1)*Ntp]
        dq = [dq_Phase[ngc:] for dq_Phase in var_xs_Phase]
        ddq = StateDiff(casadi.vertcat(*dq), var_deltaT[i], DL[tp,:])
        totalcost += var_deltaT[i]/2 * LGL_Wts[tp] * casadi.mtimes([ddq.T, R, ddq])
        
    totalcost += var_deltaT[i]

opti.minimize(totalcost)
opti.solver("ipopt") 


for t in var_deltaT:
    opti.set_initial(t, t_initial)
for t in range(T):
    opti.set_initial(var_xs[t], ini_x[t])
    opti.set_initial(var_us[t], ini_u[t])

In [8]:
try:
    sol = opti.solve_limited()
    sol_xs = [opti.value(var_x) for var_x in var_xs]
    sol_us = [opti.value(var_u) for var_u in var_us]
    sol_T = [opti.value(var_t) for var_t in var_deltaT]
except:
    print("ERROR in convergence, plotting debug info.")
    sol_xs = [opti.debug.value(var_x) for var_x in var_xs]
    sol_us = [opti.debug.value(var_u) for var_u in var_us]



******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.16, running with linear solver MUMPS 5.7.2.

Number of nonzeros in equality constraint Jacobian...:     5256
Number of nonzeros in inequality constraint Jacobian.:     4042
Number of nonzeros in Lagrangian Hessian.............:     5882

Total number of variables............................:      362
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:      306
Total number of inequality c

In [ ]:
print(sol_T)

for t in range(T):
    print(f"Collision Distance {t}: {get_obstacle_distances(sol_xs[t][:ngc])}, \n State:{Robot.collision_status(sol_xs[t][:ngc], obs.env_spheres, obs.robot_spheres)}")